In [47]:
import pandas as pd

dataset_name = "Samsum"
# Treatment
df_deepseek_1_5b = pd.read_csv(f"./{dataset_name}/Treatment/results_deepseek-ai_DeepSeek-R1-Distill-Qwen-1.5B_{dataset_name}.csv")

df_gpt_neo_125m = pd.read_csv(f"./{dataset_name}/Treatment/results_EleutherAI_gpt-neo-125m_{dataset_name}.csv")
df_gpt_neo_1_3b = pd.read_csv(f"./{dataset_name}/Treatment/results_EleutherAI_gpt-neo-1.3B_{dataset_name}.csv")
df_gpt_neo_2_7b = pd.read_csv(f"./{dataset_name}/Treatment/results_EleutherAI_gpt-neo-2.7B_{dataset_name}.csv")

df_opt_125m = pd.read_csv(f"./{dataset_name}/Treatment/results_facebook_opt-125m_{dataset_name}.csv")
df_opt_1_3b = pd.read_csv(f"./{dataset_name}/Treatment/results_facebook_opt-1.3b_{dataset_name}.csv")
# df_opt_350m = pd.read_csv(f"./{dataset_name}/Treatment/results_facebook_opt-350m_{dataset_name}.csv")
df_opt_2_7b = pd.read_csv(f"./{dataset_name}/Treatment/results_facebook_opt-2.7b_{dataset_name}.csv")
df_opt_6_7b = pd.read_csv(f"./{dataset_name}/Treatment/results_facebook_opt-6.7b_{dataset_name}.csv")

df_vicuna_7b = pd.read_csv(f"./{dataset_name}/Treatment/results_lmsys_vicuna-7b-v1.5_{dataset_name}.csv")

# Control
df_smol_1_7b = pd.read_csv(f"./{dataset_name}/Control/results_HuggingFaceTB_SmolLM2-1.7B-Instruct_{dataset_name}.csv")
df_smol_135m = pd.read_csv(f"./{dataset_name}/Control/results_HuggingFaceTB_SmolLM2-135M-Instruct_{dataset_name}.csv")
df_smol_360m = pd.read_csv(f"./{dataset_name}/Control/results_HuggingFaceTB_SmolLM2-360M-Instruct_{dataset_name}.csv")

df_phi_3_5 = pd.read_csv(f"./{dataset_name}/Control/results_microsoft_Phi-3.5-mini-instruct_{dataset_name}.csv")


In [48]:
dfs = {
    "df_deepseek_1_5b": df_deepseek_1_5b,
    "df_gpt_neo_1_3b": df_gpt_neo_1_3b,
    "df_gpt_neo_125m": df_gpt_neo_125m,
    "df_gpt_neo_2_7b": df_gpt_neo_2_7b,
    "df_opt_1_3b": df_opt_1_3b,
    "df_opt_125m": df_opt_125m,
    "df_opt_2_7b": df_opt_2_7b,
    # "df_opt_350m": df_opt_350m,
    "df_opt_6_7b": df_opt_6_7b,
    "df_vicuna_7b": df_vicuna_7b,
    "df_smol_1_7b": df_smol_1_7b,
    "df_smol_135m": df_smol_135m,
    "df_smol_360m": df_smol_360m,
    "df_phi_3_5": df_phi_3_5
}

In [49]:
# df_deepseek_1_5b.head()
len(df_gpt_neo_1_3b)

819

In [19]:
from scipy.stats import ttest_ind

# Perform Welch’s t-test; return a dictionary mapping score names to significance (1 or 0)
def t_test(df_suspicious_name, df_control_name, score_names):
    df_suspicious, df_control = dfs[df_suspicious_name], dfs[df_control_name]
    score_sig = dict()
    for score_name in score_names:
        scores_suspicious, scores_control = df_suspicious[score_name], df_control[score_name]
        stat, p_value = ttest_ind(scores_suspicious, scores_control, equal_var=False)
        print(f"comparing {score_name} between {df_suspicious_name} and {df_control_name}")
        print(f"mean ({df_suspicious_name}) = {scores_suspicious.mean()}, mean ({df_control_name}) = {scores_control.mean()}")
        print(f"t-statistic: {stat:.3f}, p-value: {p_value:.5f}")
        if p_value <= 0.05:
            print("Significant difference between suspicious and control groups.")
            score_sig[score_name] = 1
        else:
            print("No significant difference detected.")
            score_sig[score_name] = 0
        print("----------------------------------------------------------------")
    return score_sig


In [20]:
score_names = df_deepseek_1_5b.drop(['Unnamed: 0', 'Model', 'Task_Prefix', 'Dataset_Name', 'Model_Responses', 'Gold_Labels'], axis=1).columns
score_names = list(score_names)

In [22]:
import itertools
# all_pairs = list(itertools.product(treatment_dfs.keys(), control_dfs.keys()))

In [50]:
gpt = ["df_gpt_neo_125m", "df_gpt_neo_1_3b", "df_gpt_neo_2_7b"]
# opt = ["df_opt_125m", "df_opt_350m", "df_opt_1_3b", "df_opt_2_7b", "df_opt_6_7b"]
opt = ["df_opt_125m", "df_opt_1_3b", "df_opt_2_7b", "df_opt_6_7b"]
vicuna = ["df_vicuna_7b"]
smol = ["df_smol_135m", "df_smol_360m", "df_smol_1_7b"]
phi = ["df_phi_3_5"]
treatment = gpt + opt + vicuna
control = smol + phi

In [51]:
# doing t-test for every possible combination 
all_pairs = list(itertools.product(treatment, control))

total = 0 # count total model pairs
model_pair_analysis = [] # store all model pairs with resulting score indications

for suspicious, control in all_pairs: 
    model_pair_analysis.append((
        suspicious,
        control,
        t_test(suspicious, control, score_names)
    ))
    total+= 1

comparing cos_sim_scores between df_gpt_neo_125m and df_smol_135m
mean (df_gpt_neo_125m) = 0.46766190380608536, mean (df_smol_135m) = 0.5515517186264306
t-statistic: -10.008, p-value: 0.00000
Significant difference between suspicious and control groups.
----------------------------------------------------------------
comparing levenshtein_distance between df_gpt_neo_125m and df_smol_135m
mean (df_gpt_neo_125m) = 90.24908424908425, mean (df_smol_135m) = 88.08669108669109
t-statistic: 0.884, p-value: 0.37689
No significant difference detected.
----------------------------------------------------------------
comparing rouge1_precision between df_gpt_neo_125m and df_smol_135m
mean (df_gpt_neo_125m) = 0.14838815422636537, mean (df_smol_135m) = 0.22780473380558505
t-statistic: -11.137, p-value: 0.00000
Significant difference between suspicious and control groups.
----------------------------------------------------------------
comparing rouge1_recall between df_gpt_neo_125m and df_smol_135m


In [59]:
from collections import defaultdict

score_proportion = defaultdict(int) # store count of total model pairs where score S is significantly difference
score_sig_match = defaultdict(list) # store pairs that contribute to the counts above^

for sus, con, vals in model_pair_analysis:
    for m in vals.keys(): 
        score_proportion[m] += vals[m]
        if vals[m]:
            score_sig_match[m].append((sus, con))

In [61]:
sco

defaultdict(list,
            {'cos_sim_scores': [('df_gpt_neo_125m', 'df_smol_135m'),
              ('df_gpt_neo_125m', 'df_smol_360m'),
              ('df_gpt_neo_125m', 'df_smol_1_7b'),
              ('df_gpt_neo_125m', 'df_phi_3_5'),
              ('df_gpt_neo_1_3b', 'df_smol_135m'),
              ('df_gpt_neo_1_3b', 'df_smol_360m'),
              ('df_gpt_neo_1_3b', 'df_smol_1_7b'),
              ('df_gpt_neo_1_3b', 'df_phi_3_5'),
              ('df_gpt_neo_2_7b', 'df_smol_135m'),
              ('df_gpt_neo_2_7b', 'df_smol_360m'),
              ('df_gpt_neo_2_7b', 'df_smol_1_7b'),
              ('df_gpt_neo_2_7b', 'df_phi_3_5'),
              ('df_opt_125m', 'df_smol_135m'),
              ('df_opt_125m', 'df_smol_360m'),
              ('df_opt_125m', 'df_smol_1_7b'),
              ('df_opt_125m', 'df_phi_3_5'),
              ('df_opt_1_3b', 'df_smol_135m'),
              ('df_opt_1_3b', 'df_smol_360m'),
              ('df_opt_1_3b', 'df_smol_1_7b'),
              ('df_opt_1_3b', '

In [27]:
t_test("df_opt_2_7b", "df_smol_1_7b", score_names)

comparing cos_sim_scores between df_opt_2_7b and df_smol_1_7b
mean (df_opt_2_7b) = 0.35611472660438265, mean (df_smol_1_7b) = 0.6813394640901004
t-statistic: -39.139, p-value: 0.00000
Significant difference between suspicious and control groups.
----------------------------------------------------------------
comparing levenshtein_distance between df_opt_2_7b and df_smol_1_7b
mean (df_opt_2_7b) = 90.17094017094017, mean (df_smol_1_7b) = 85.94505494505495
t-statistic: 1.883, p-value: 0.05989
No significant difference detected.
----------------------------------------------------------------
comparing rouge1_precision between df_opt_2_7b and df_smol_1_7b
mean (df_opt_2_7b) = 0.1283207980550599, mean (df_smol_1_7b) = 0.3448629078307945
t-statistic: -29.058, p-value: 0.00000
Significant difference between suspicious and control groups.
----------------------------------------------------------------
comparing rouge1_recall between df_opt_2_7b and df_smol_1_7b
mean (df_opt_2_7b) = 0.1907957

{'cos_sim_scores': 1,
 'levenshtein_distance': 0,
 'rouge1_precision': 1,
 'rouge1_recall': 1,
 'rouge1_f1': 1,
 'rouge2_precision': 1,
 'rouge2_recall': 1,
 'rouge2_f1': 1,
 'rougeL_precision': 1,
 'rougeL_recall': 1,
 'rougeL_f1': 1,
 'Min_10.0%_Prob': 0,
 'Min_20.0%_Prob': 0,
 'Min_30.0%_Prob': 0,
 'Min_40.0%_Prob': 0,
 'Min_50.0%_Prob': 0}

In [28]:
t_test("df_gpt_neo_125m", "df_smol_135m", score_names)

comparing cos_sim_scores between df_gpt_neo_125m and df_smol_135m
mean (df_gpt_neo_125m) = 0.46766190380608536, mean (df_smol_135m) = 0.5515517186264306
t-statistic: -10.008, p-value: 0.00000
Significant difference between suspicious and control groups.
----------------------------------------------------------------
comparing levenshtein_distance between df_gpt_neo_125m and df_smol_135m
mean (df_gpt_neo_125m) = 90.24908424908425, mean (df_smol_135m) = 88.08669108669109
t-statistic: 0.884, p-value: 0.37689
No significant difference detected.
----------------------------------------------------------------
comparing rouge1_precision between df_gpt_neo_125m and df_smol_135m
mean (df_gpt_neo_125m) = 0.14838815422636537, mean (df_smol_135m) = 0.22780473380558505
t-statistic: -11.137, p-value: 0.00000
Significant difference between suspicious and control groups.
----------------------------------------------------------------
comparing rouge1_recall between df_gpt_neo_125m and df_smol_135m


{'cos_sim_scores': 1,
 'levenshtein_distance': 0,
 'rouge1_precision': 1,
 'rouge1_recall': 1,
 'rouge1_f1': 1,
 'rouge2_precision': 1,
 'rouge2_recall': 1,
 'rouge2_f1': 1,
 'rougeL_precision': 1,
 'rougeL_recall': 1,
 'rougeL_f1': 1,
 'Min_10.0%_Prob': 0,
 'Min_20.0%_Prob': 0,
 'Min_30.0%_Prob': 1,
 'Min_40.0%_Prob': 1,
 'Min_50.0%_Prob': 1}

In [29]:
t_test("df_opt_125m", "df_smol_135m", score_names)

comparing cos_sim_scores between df_opt_125m and df_smol_135m
mean (df_opt_125m) = 0.3881366546271467, mean (df_smol_135m) = 0.5515517186264306
t-statistic: -17.824, p-value: 0.00000
Significant difference between suspicious and control groups.
----------------------------------------------------------------
comparing levenshtein_distance between df_opt_125m and df_smol_135m
mean (df_opt_125m) = 89.27350427350427, mean (df_smol_135m) = 88.08669108669109
t-statistic: 0.492, p-value: 0.62281
No significant difference detected.
----------------------------------------------------------------
comparing rouge1_precision between df_opt_125m and df_smol_135m
mean (df_opt_125m) = 0.1268978302344526, mean (df_smol_135m) = 0.22780473380558505
t-statistic: -14.088, p-value: 0.00000
Significant difference between suspicious and control groups.
----------------------------------------------------------------
comparing rouge1_recall between df_opt_125m and df_smol_135m
mean (df_opt_125m) = 0.1741281

{'cos_sim_scores': 1,
 'levenshtein_distance': 0,
 'rouge1_precision': 1,
 'rouge1_recall': 1,
 'rouge1_f1': 1,
 'rouge2_precision': 1,
 'rouge2_recall': 1,
 'rouge2_f1': 1,
 'rougeL_precision': 1,
 'rougeL_recall': 1,
 'rougeL_f1': 1,
 'Min_10.0%_Prob': 0,
 'Min_20.0%_Prob': 0,
 'Min_30.0%_Prob': 1,
 'Min_40.0%_Prob': 1,
 'Min_50.0%_Prob': 1}